In [56]:
import numpy as np

def get_target(y_data) :
    
    y_values = []
    
    for item in y_data:
        
        if item not in y_values:
            y_values.append(item)
            
    return y_values

In [57]:
def getFrequency(y_data, label) :
    
    length = len(y_data)
    target = get_target(y_data)
    frequency = 0 
    matching = 0
        
    for item in y_data :
            
        if label == item :
                
            matching=matching+1
                
    frequency = matching/length
    
    return frequency           

In [58]:
import math

def getEntropy(y_data) :
    
    entropy = 0
    target = get_target(y_data)
    
    for tg in target :
        
        f = getFrequency(y_data,tg)
        entropy = entropy - f*math.log(f,2)
        
    return entropy

In [59]:
import numpy as np

def getProbability(column) :
    
    length = len(column)
    target = get_target(column)
    probability = []
    
    for tg in target :
        matching = 0
        for item in column :
            
            if tg == item :
                matching=matching+1
                
            prob = matching/length
        probability.append(prob)
    
    return probability

In [60]:
def getConditionalEntropy(column,feature_value,y_data) :
        
        new_Y = []
        conditionalEntropy=0
        a=0
        
        for item in column :
                        
            if feature_value == item :
                
                new_Y.append(y_data[a])

            a=a+1 
                
        probability = np.array(getProbability(new_Y)) 
        
        for p in probability :
    
            conditionalEntropy = conditionalEntropy-p*math.log(p,2)
        
        return conditionalEntropy   

In [61]:
def getInformationGain(column,y_data):
    
    probabilities = getProbability(column)
    entropies = []
    values=get_target(column)
    informationGain=0
    conditionalEntropy=0
    
    for item in values:
        
        x=getConditionalEntropy(column,item,y_data)
        entropies.append(x)
        
    a=0
    for el in probabilities:
        
        conditionalEntropy = conditionalEntropy + (probabilities[a]*entropies[a])
        a=a+1
        
    informationGain = getEntropy(y_data) - conditionalEntropy
    
    return informationGain

In [62]:
def computeBestSplit(x_data,y_data,feature_names):
    
    ig=[]
    a=0
    for column in x_data.T:
        ig.append(getInformationGain(column,y_data))
        a=a+1
        
        
    return feature_names[np.argmax(ig)],np.argmax(ig)

In [63]:
def getInducedSubDatasets (x_data,y_data,bestSplit):
    
    xT_data=x_data.T
    target=get_target(xT_data[bestSplit[1]])
    
    inducedSubDatsets_x = []
    inducedSubDatsets_y = []
    
    for tg in target :
        
        temp_x = []
        temp_y = []
        
        a=0
        for item in x_data :
            
            if tg==item[0] :
                
                temp_x.append(x_data[a])
                temp_y.append(y_data[a])
                
            a=a+1
            
        inducedSubDatsets_x.append(temp_x)
        inducedSubDatsets_y.append(temp_y)
        
    return np.array(inducedSubDatsets_x),np.array(inducedSubDatsets_y)   

In [64]:
def pure(y_data):
    
    first=y_data[0]
    pure= True
    for item in y_data:
        
        if item!=first:
            pure= False
        
    return pure

In [65]:
def buildTree(x_data,y_data,feature_names):
    
    tree = []
    
    if pure(y_data): 
        return y_data[0]
    
    best_feature = computeBestSplit(x_data,y_data,feature_names)
    
    root=best_feature
    
    tree.append(root)
    
    subdataset=np.array(getInducedSubDatasets(x_data,y_data,best_feature))
    
    a=0
    for item in subdataset[0] :
        
        x=np.array(subdataset.T[a,0])
        y=np.array(subdataset.T[a,1])
        
        
        if pure(y): 
            tree.append(y[0])
        
        else: 
            c = computeBestSplit(x,y,feature_names)
            tree.append(c)
              
        a=a+1
        
    return tree

In [66]:
import numpy as np

X_data= np.array ([['Sunny','Hot','High','Weak'],
        ['Sunny','Hot','High','Strong'],
        ['Overcast','Hot','High','Weak'],
        ['Rain','Mild','High','Weak'],
        ['Rain','Cool','Normal','Weak'],
        ['Rain','Cool','Normal','Strong'],
        ['Overcast','Cool','Normal','Strong'],
        ['Sunny','Mild','High','Weak'],
        ['Sunny','Cool','Normal','Weak'],
        ['Rain','Mild','Normal','Weak'],
        ['Sunny','Mild','Normal','Strong'],
        ['Overcast','Mild','High','Strong'],
        ['Overcast','Hot','Normal','Weak'],
        ['Rain','Mild','High','Strong']])

y_data=np.array(['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no'])

feature_names=np.array(['outlook','temperature','humidity','wind'])

print(buildTree(X_data,y_data,feature_names))

[('outlook', 0), ('humidity', 2), 'yes', ('wind', 3)]
